# Competitiveness Analysis (by Country and Product)

MapReduce version to save memory.

In [1]:
import os
import pandas as pd

src = 'un'

if src == 'itc':
    path = '//172.20.23.190/ds/Raw Data/2016大數爬蟲案/data/ITC HS6/all/'
    files = pd.Series(os.listdir(path))
    # Filter for import data
    files = files[files.str.contains('_I')]
    # Exclude Taiwan and world total from importing countries
    files.drop(files[files.str.contains('Taipei|All_')].index.values, inplace=True)
else:
    path = '//172.26.1.102/dstore/uncomtrade/annual_reduced/'
    files = ['un-import-hs6-shiny-2012-2015.csv']

In [2]:
# Load product description file
desc = pd.read_csv('//172.26.1.102/dstore/Projects/mof-crawler/full_hscode11.tsv', sep='\t',
                   dtype='str', usecols=['hs2cn', 'hs4cn', 'hs6', 'hs6cn'])
desc.columns = ['desc2', 'desc4', 'product', 'desc6']
# Because each row corresponds to an HS11 code in the original table, need to remove duplicates
desc.drop_duplicates(subset='product', inplace=True)

In [3]:
# Country name mapping table
ctry_map = pd.read_csv('//172.20.23.190/ds/Raw Data/2016大數爬蟲案/data/ITC HS6/itc_df_complete.csv',
                       usecols=[dict(itc='itc_name', un='ds_code')[src], 'countryName'],
                       dtype=dict(itc=None, un={'ds_code': str})[src])
ctry_map.columns = ['country', 'ch_name']
# Convert to en -> zh dictionary (UN: ISO 3166-1 numeric code -> zh)
ctry_map = ctry_map.set_index('country').to_dict()['ch_name']

In [4]:
# Company export data
com_ex = pd.read_csv('C:/Users/2093/Desktop/Data Center/03. Data/06. companies/財政部廠商進出口資料/KMG_HS6COUNTRY.csv',
                     names=['ban', 'code', 'country', 'year', 'month', 'ex', 'im'], header=0,
                     dtype={'ban': str, 'code': str, 'country': str, 'year': str, 'month': str,
                            'ex': int, 'im': int})
com_ex['code'] = com_ex['code'].str.zfill(6)
# Remove yearly total rows
com_ex = com_ex[com_ex['month'].notnull()]
# Pad zeros and construct DatetimeIndex
com_ex['month'] = com_ex['month'].apply(lambda x: x.zfill(2))
com_ex.index = pd.to_datetime(com_ex['year'] + com_ex['month'], format='%Y%m')
com_ex.index.name = 'date'
# Drop original year and month columns
com_ex = com_ex.drop(['year', 'month'], axis=1)

# Extract 2015 data and sum for yearly total
com_ex = com_ex['2015'].groupby(['ban', 'code', 'country']).sum().reset_index()
# Filter out zero entries
com_ex = com_ex[com_ex['ex'] != 0]
# Alternatively, for each commodity, keep only companies with over 100 thousand USD worth of
# yearly export
# sizable = com_ex.groupby(['ban', 'code']).sum().reset_index().query('ex != 0')['ban'].unique()
# com_ex = com_ex[com_ex['ban'].isin(sizable)]

# Company profile datasets
path_crm = 'C:/Users/2093/Desktop/Data Center/03. Data/05. TAITRA/CRM/'
tax = pd.read_csv(path_crm + 'tax_utf-8.csv', header=0, index_col=False,
                  names=['bogus', 'address', 'id', 'com_name', 'capital', 'est_date', 'invoice',
                         'biz1_id', 'biz1', 'biz2_id', 'biz2', 'biz3_id', 'biz3', 'biz4_id', 'biz4'],
                  dtype={'bogus': str,
                         'address': str,
                         'id': str,
                         'com_name': str,
                         'capital': int,
                         'est_date': int,
                         'invoice': str,
                         'biz1_id': str,
                         'biz1': str,
                         'biz2_id': str,
                         'biz2': str,
                         'biz3_id': str,
                         'biz3': str,
                         'biz4_id': str,
                         'biz4': str})
biz_nature = pd.read_csv('C:/Users/2093/Desktop/Data Center/03. Data/06. companies/COMP_TYPE.csv',
                         index_col=False, header=0,
                         names=['biz_nature', 'biz_nature_id', 'id', 'user', 'date', 'bogus'],
                         dtype={'biz_nature': str,
                                'biz_nature_id': int,
                                'id': str,
                                'user': str,
                                'date': str,
                                'bogus': str})
# List of Taiwan exporters
exporter = biz_nature.loc[biz_nature['biz_nature'] == 'Exporter', 'id']
# Event participation data
event = pd.read_csv(path_crm + 'crm_group.csv', usecols=range(4), header=0,
                    names=['year', 'event_name', 'source', 'event_id'])
attend = pd.read_csv(path_crm + 'crm_basic_group.csv', usecols=range(2), header=0,
                     names=['event_id', 'bogus'])
# Companies that have participated in TAITRA event at least once within 2013-2016
participant = attend.merge(event[['year', 'event_id']], on='event_id').merge(
    tax[['bogus', 'id']], on='bogus').query('year > 2013')['id'].unique()

# ISO 3166-1 Alpha-2 to zh name mapping
alpha_map = pd.read_csv('C:/Users/2093/Desktop/Data Center/03. Data/03. import_export/ds_export_market_framework.csv',
                        usecols=['mof_enCode', 'export_countryName'])
alpha_map.columns = ['alpha2', 'ch_name']
# Convert to Alpha-2 -> zh dictionary
alpha_map = alpha_map.set_index('alpha2').to_dict()['ch_name']

# Alternative: ISO 3166-1 Alpha-2 to numeric code mapping
# alpha_map = pd.read_csv('C:/Users/2093/Desktop/Data Center/03. Data/03. import_export/ds_export_market_framework.csv',
#                         usecols=['mof_enCode', 'iso_numCode'], dtype=str)
# alpha_map.columns = ['alpha2', 'num']
# alpha_map['num'] = alpha_map['num'].str.replace('^0+', '')
# alpha_map = alpha_map.set_index('alpha2').to_dict()['num']

# For each country and commodity, compute no. of exporters and among them, those that have
# participated in TRATRA events
n_com = com_ex[com_ex['ban'].isin(exporter)].assign(
    is_par = lambda x: x['ban'].isin(participant)).groupby(['country', 'code']).agg(
    {'ban': 'count', 'is_par': sum}).reset_index().rename(
    columns=dict(code='product', ban='n_exporter'))
n_com['country'].replace(alpha_map, inplace=True)
n_com['is_par'] = n_com['is_par'].astype(int)

In [5]:
from scipy.stats import kurtosis
import numpy as np
from functools import reduce

def aggr_data(file):
    
    if src == 'itc':
        df = pd.read_csv(path + file, index_col=0,
                         dtype={'Country': 'object',
                                'Product Code': 'object',
                                'Partner': 'object',
                                'Value in 2001': 'float',
                                'Value in 2002': 'float',
                                'Value in 2003': 'float',
                                'Value in 2004': 'float',
                                'Value in 2005': 'float',
                                'Value in 2006': 'float',
                                'Value in 2007': 'float',
                                'Value in 2008': 'float',
                                'Value in 2009': 'float',
                                'Value in 2010': 'float',
                                'Value in 2011': 'float',
                                'Value in 2012': 'float',
                                'Value in 2013': 'float',
                                'Value in 2014': 'float',
                                'Value in 2015': 'float'}).reset_index(drop=True)
    else:
        df = pd.read_csv(path + file,
                         dtype={'Country': 'object',
                                'Product Code': 'object',
                                'Partner': 'object',
                                'Value in 2012': 'float',
                                'Value in 2013': 'float',
                                'Value in 2014': 'float',
                                'Value in 2015': 'float'}).loc[
            :, ['Country', 'Product Code', 'Partner',
                'Value in 2012', 'Value in 2013', 'Value in 2014', 'Value in 2015']]

    # Remove the leading single quote (') in product code column
    df['Product Code'] = df['Product Code'].apply(lambda x: x[1:])
    # If UN, exclude Taiwan ('490') and world total ('all') from importing countries
    df = df[~df['Country'].str.contains('490|all')]
    # Remove rows for commodities sum
    df = df[df['Product Code'] != 'TOTAL']
    # Remove rows where partner is 'All' (it seems that HS6 tables don't have this code)
    # In UN's case, this becomes 'all' and '0' (the latter stands for World)
    df = df[~df['Partner'].str.contains('All|all|^0$')]
    # Select only columns for 2012 to 2015
    df = pd.concat((df.loc[:, :'Partner'], df.loc[:, 'Value in 2012':]), axis=1)
    df.columns = ['country', 'product', 'partner', 'val12', 'val13', 'val14', 'val15']
    # Compute growth rates
    def growthRate(data, start_year, end_year):
        return ((data['val' + str(end_year)] - data['val' + str(start_year)])
                / data['val' + str(start_year)] * 100)
    df['g13'] = growthRate(df, 12, 13)
    df['g14'] = growthRate(df, 13, 14)
    df['g15'] = growthRate(df, 14, 15)

    # Compute total imports for all (country, product) pairs
    total = df.groupby(['country', 'product']).agg({
            'val12': 'sum',
            'val13': 'sum',
            'val14': 'sum',
            'val15': 'sum'})
    total['g13'] = growthRate(total, 12, 13)
    total['g14'] = growthRate(total, 13, 14)
    total['g15'] = growthRate(total, 14, 15)
    total = total[['val15', 'g13', 'g14', 'g15']].reset_index()

    # Compute commodity-wise market share for each partner country
    df['share'] = df['val15'] / df.groupby(['country', 'product'])['val15'].transform('sum') * 100
    # Compute commodity-wise rank for each partner country
    df['rank'] = df.groupby(['country', 'product'])['val15'].rank(ascending=False, method='min')

    # Compute no. of non-zero partners for each importing country by commodity
    n_partner = df[(df['val15'] != 0) & (df['val15'].notnull())].groupby(
        ['country', 'product']).agg({'partner': 'count'}).rename(columns={'partner': 'n_partner'})
    # Compute excess kurtosis for each country by commodity
    kurtos = df.groupby(['country', 'product']).agg(
        {'val15': lambda x: kurtosis(x, nan_policy='omit')}).rename(columns={'val15': 'kurtos'})
    # Compute Pearson's median skewness coefficient for each country by commodity
    skewness = df.groupby(['country', 'product']).agg(
        {'val15': lambda x: 3 * (x.mean() - x.median()) / x.std() if x.std() != 0 else np.nan}).rename(
        columns={'val15': 'skew'})

    # Extract data for Taiwan (UN: '490')
    tw = df.loc[df['partner'].str.contains('Taipei, Chinese|490'),
                ['country', 'product', 'val15', 'g13', 'g14', 'g15', 'share', 'rank']]
    tw.columns = ['country', 'product', 'tw_val15', 'tw_g13', 'tw_g14', 'tw_g15',
                  'tw_share', 'tw_rank']
    # When import value from Taiwan is zero, manually overwrite corresponding rank of Taiwan with NaN
    tw.loc[tw['tw_val15'] == 0, 'tw_rank'] = None

    # Extract data for top 3
    top3 = df.groupby(['country', 'product']).apply(lambda x: x.nsmallest(3, 'rank')).loc[
        :, ['country', 'product', 'partner', 'val15', 'g13', 'g14', 'g15', 'share']]
    def getCountryByRank(data, rank):
        rs = data.groupby(['country', 'product']).nth(rank).reset_index().loc[
        :, ['country', 'product', 'partner', 'val15', 'g13', 'g14', 'g15', 'share']]
        rs.columns = (['country', 'product', 'partner']
                      + [str(rank + 1) + '_' + x for x in ['val15', 'g13', 'g14', 'g15', 'share']])
        return rs
    first  = getCountryByRank(top3, 0)
    second = getCountryByRank(top3, 1)
    third  = getCountryByRank(top3, 2)

    # Merge all tables
    rs = total.merge(n_partner, how='left', left_on=['country', 'product'], right_index=True).merge(
        kurtos, how='left', left_on=['country', 'product'], right_index=True).merge(
        skewness, how='left', left_on=['country', 'product'], right_index=True).merge(
        #n_com, how='left', on=['country', 'product']).merge(
        tw, how='left', on=['country', 'product']).merge(
        first, how='left', on=['country', 'product']).rename(columns={'partner': '1_name'}).merge(
        second, how='left', on=['country', 'product']).rename(columns={'partner': '2_name'}).merge(
        third, how='left', on=['country', 'product']).rename(columns={'partner': '3_name'}).merge(
        desc, how='left', on='product').iloc[:, [0, 1, -3, -2, -1] + list(range(2, 33))]
    
    # Replace en country names (UN: ISO 3166-1 numeric code) with zh names
    for col in ['country', '1_name', '2_name', '3_name']:
        if src == 'itc':
            rs[col].replace(ctry_map, inplace=True)
        else:
            rs[col] = rs[col].str.zfill(3).replace(ctry_map)

    # Merge with n_com on zh names
    rs = rs.merge(n_com, how='left', on=['country', 'product']).iloc[
        :, list(range(12)) + [-2, -1] + list(range(12, 36))]
        
    return rs

In [6]:
%%time

df_map = map(lambda f: aggr_data(f), files)
df = reduce(lambda x, y: pd.concat([x, y], axis=0, ignore_index=True), df_map)

# Output results
df.to_csv('comp_aggregate_6.csv', sep=',', index=False)
# Output HS code table
df[['product', 'desc2', 'desc4', 'desc6']].drop_duplicates().sort_values('product').to_csv(
    'hs_table.csv', sep=',', index=False)
# Separate tables
for hs2 in df['product'].str.slice(0, 2).unique():
    df[df['product'].str.contains('^{}'.format(hs2))].to_csv(
        'comp_aggregate_6_{}.csv'.format(hs2), sep=',', index=False)

C:\Users\2093\AppData\Local\Continuum\Anaconda3\lib\site-packages\numpy\ma\core.py:4185: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


Wall time: 32min 4s


In [7]:
# Output no. of Taiwan exporters by commodity only
n_com_p = com_ex[com_ex['ban'].isin(exporter)].assign(
    is_par = lambda x: x['ban'].isin(participant)).drop_duplicates(['ban', 'code']).groupby('code').agg(
    {'ban': 'count', 'is_par': sum}).reset_index().rename(
    columns=dict(code='product', ban='n_exporter'))
n_com_p['par_rate'] = 100 * n_com_p['is_par'] / n_com_p['n_exporter']
n_com_p['is_par'] = n_com_p['is_par'].astype(int)
n_com_p.to_csv('n_com_p.csv', sep=',', index=False)